
# Consumer Choice — Multinomial Logit (MNL) — Python Notebook

**When to Use**  
- Modeling **discrete choices** among competing alternatives (e.g., brand selection, plan tiers, product variants).  
- Estimating **price sensitivity**, **feature effects**, and **market shares** for scenario planning.  

**Best Application**  
- Assortment/pricing simulations where each decision maker selects **one** option from a mutually exclusive set.  
- Conjoint/choice experiments and **share-of-preference** simulations.  

**When Not to Use**  
- When the **IIA (Independence of Irrelevant Alternatives)** assumption is likely violated (e.g., close substitutes / nested categories). Consider **nested** or **mixed logit**.  
- When choices are **multiple (non-mutually exclusive)** or involve **quantity** decisions—use other discrete–continuous or count models.

**How to Interpret Results**  
- Coefficients enter **systematic utility**; signs indicate **direction** (e.g., negative for price).  
- Exponentiated coefficients relate to **odds** between alternatives; changes in attributes shift **choice probabilities**.  
- Simulate **elasticities** and **counterfactual shares** by altering attributes (prices, promos) and recomputing probabilities.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

pd.set_option('display.max_columns', 120)
plt.rcParams['figure.figsize'] = (8,4)
rng = np.random.default_rng(7)


### Data: Synthetic consumer choices among 3 brands with prices and promo features

In [ ]:

n = 1500                      # number of consumers (trials)
brands = ['A','B','C']        # three alternatives
J = len(brands)

# Generate individual-level price and a binary promo feature per brand
base_price = {'A': 5.00, 'B': 4.50, 'C': 4.00}
price = {b: base_price[b] + rng.normal(0, 0.4, n) for b in brands}
promo = {b: rng.binomial(1, 0.25, n) for b in brands}   # e.g., shelf display/coupon

# True utilities: brand-specific price sensitivity + promo lift
beta_price = {'A': -0.9, 'B': -0.8, 'C': -0.7}
beta_promo = {'A': 0.5, 'B': 0.6, 'C': 0.4}
beta_brand_const = {'A': 0.4, 'B': 0.2, 'C': 0.0}   # brand preference constants

U = np.zeros((n, J))
for j, b in enumerate(brands):
    U[:, j] = (beta_brand_const[b]
               + beta_price[b] * price[b]
               + beta_promo[b] * promo[b]
               + rng.normal(0, 1, n))

choice_idx = U.argmax(axis=1)
choice = np.array(brands)[choice_idx]

# Assemble long-format dataset for modeling
rows = []
for i in range(n):
    for j, b in enumerate(brands):
        rows.append({
            'obs': i,
            'brand': b,
            'choice': int(choice[i] == b),
            'price': float(price[b][i]),
            'promo': int(promo[b][i]),
        })

df_long = pd.DataFrame(rows)
df_long.head(6)


### Model Matrix: Brand dummies with brand-specific price and promo effects

In [ ]:

# Reference alternative: set brand C as the base (no explicit dummy needed)
df_long = df_long.assign(
    brand_A = (df_long['brand'] == 'A').astype(int),
    brand_B = (df_long['brand'] == 'B').astype(int)
)

# Interactions to allow brand-specific price/promo effects
df_long['price_A'] = df_long['price'] * df_long['brand_A']
df_long['price_B'] = df_long['price'] * df_long['brand_B']
df_long['price_C'] = df_long['price'] * (1 - df_long['brand_A'] - df_long['brand_B'])

df_long['promo_A'] = df_long['promo'] * df_long['brand_A']
df_long['promo_B'] = df_long['promo'] * df_long['brand_B']
df_long['promo_C'] = df_long['promo'] * (1 - df_long['brand_A'] - df_long['brand_B'])

# Target and features
y = df_long['choice']
X = df_long[['brand_A','brand_B','price_A','price_B','price_C','promo_A','promo_B','promo_C']]
X = sm.add_constant(X, prepend=True)


### Fit Multinomial Logit (base = Brand C)

In [ ]:

mnl = sm.MNLogit(y, X)
res = mnl.fit(method='newton', maxiter=200, disp=False)
print(res.summary())


### In-Sample Predicted Shares

In [ ]:

pred_probs = res.predict(X)  # columns correspond to outcomes 0/1 for "choice"; we need to reshape
# For MNLogit with a binary y in long format, predictions map to P(choice=1) for each row.
df_long['p_choose'] = pred_probs.values.flatten()

# Aggregate to market share per brand
pred_share = df_long.groupby('brand')['p_choose'].mean().rename('pred_share')
obs_share = df_long.loc[df_long['choice']==1,'brand'].value_counts(normalize=True).rename('obs_share')
shares = pd.concat([obs_share, pred_share], axis=1)
shares


### Simulate Own/Cross Price Elasticities

In [ ]:

def simulate_shares(df, model, Xcols):
    Xsim = sm.add_constant(df[Xcols], prepend=True)
    p = model.predict(Xsim).values.flatten()
    tmp = df.copy()
    tmp['p'] = p
    return tmp.groupby('brand')['p'].mean()

Xcols = ['brand_A','brand_B','price_A','price_B','price_C','promo_A','promo_B','promo_C']

# Baseline shares
base_shares = simulate_shares(df_long, res, Xcols)

# Own-price elasticity for brand A: +1% price_A (approx by adding 1% to price for rows of brand A)
df_own = df_long.copy()
mask_A = df_own['brand']=='A'
df_own.loc[mask_A, 'price_A'] *= 1.01
shares_own = simulate_shares(df_own, res, Xcols)

elas_A = ((shares_own['A'] - base_shares['A']) / base_shares['A']) / 0.01

# Cross-price elasticity on brand B when A price increases 1%
elas_B_wrt_A = ((shares_own['B'] - base_shares['B']) / base_shares['B']) / 0.01

pd.DataFrame({
    'baseline_share': base_shares.round(4),
    'share_after_A_price+1%': shares_own.round(4)
}), float(elas_A), float(elas_B_wrt_A)


### Scenario: Brand B 10% price discount

In [ ]:

df_disc = df_long.copy()
mask_B = df_disc['brand']=='B'
df_disc.loc[mask_B,'price_B'] *= 0.90
shares_disc = simulate_shares(df_disc, res, Xcols)

pd.DataFrame({
    'baseline_share': base_shares.round(4),
    'B_price_minus_10%': shares_disc.round(4)
})


### IIA Check (Heuristic): Remove Brand C and Refit vs. Implied Odds

In [ ]:

# Compare the log-odds A vs B with and without C present.
# If IIA holds, odds(A/B) shouldn't change much when C is removed.

# Subset to A/B rows
AB = df_long[df_long['brand'].isin(['A','B'])].copy()
y_ab = AB['choice']
X_ab = AB[['brand_A','brand_B','price_A','price_B','price_C','promo_A','promo_B','promo_C']]
X_ab = sm.add_constant(X_ab, prepend=True)
mnl_ab = sm.MNLogit(y_ab, X_ab).fit(method='newton', maxiter=200, disp=False)

# Compute average log-odds across dataset (A vs B) from full model vs A/B-only model
# For simplicity, compare average predicted odds
def avg_log_odds_A_vs_B(model, df):
    # Compute probability of choosing the row's brand; separate into A and B
    Xs = sm.add_constant(df[['brand_A','brand_B','price_A','price_B','price_C','promo_A','promo_B','promo_C']], prepend=True)
    p = model.predict(Xs).values.flatten()
    tmp = df.copy()
    tmp['p'] = p
    pA = tmp.loc[tmp['brand']=='A','p'].mean()
    pB = tmp.loc[tmp['brand']=='B','p'].mean()
    return np.log(pA / pB + 1e-9)

llo_full = avg_log_odds_A_vs_B(res, AB)
llo_ab = avg_log_odds_A_vs_B(mnl_ab, AB)
delta = float(llo_ab - llo_full)
delta



---

### Practical Guidance
- If the **IIA** diagnostic is large (odds shift when an option is removed), consider **nested** or **mixed logit**.  
- Use **brand-specific** price coefficients to capture heterogeneous price sensitivity.  
- For conjoint studies, build **alternative-specific attributes** and simulate **shares** under scenarios.

### References (non-link citations)
1. Train — *Discrete Choice Methods with Simulation*.  
2. Greene — *Econometric Analysis*.  
3. Rossi, Allenby & McCulloch — *Bayesian Statistics and Marketing*.
